In [21]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6,7'

import os
import yaml
import torch

import sys
# cur_dir = os.path.split(os.path.abspath(__file__))[0]
# con_pth = cur_dir.rsplit('/',2)[0]
# sys.path.append(con_pth)
from src.models.celltrack_plmodel import CellTrackLitModel
from src.inference.graph_dataset_inference import CellTrackDataset
import warnings
# warnings.filterwarnings("ignore")

import os
import os.path as osp
import torch
import pandas as pd
import numpy as np
from skimage import io
import warnings
# warnings.filterwarnings("ignore")
import imageio

In [22]:
model_path = r"/home/sunrui/cellwork/track/cell-tracker-gnn-main/logs/runs/2023-05-24/12-53-04/checkpoints/epoch=56.ckpt"
num_seq = r"01"
output_csv = r'/data/sunrui/celldata/r03c06f04ch1/'

path_output = output_csv
second_path = num_seq
path_output_folder = os.path.join(path_output, second_path) + '_RES_inference'

In [23]:
file1 = os.path.join(path_output_folder, 'pytorch_geometric_data.pt')
file2 = os.path.join(path_output_folder, 'all_data_df.csv')
file3 = os.path.join(path_output_folder, 'raw_output.pt')

In [24]:
df_data = pd.read_csv(file2)
test_data = torch.load(file1)
outputs = torch.load(file3)

In [29]:
print(test_data)
print(outputs)

Data(x=[16172, 13], edge_index=[2, 241757], x_2=[16172, 128], edge_feat=[241757, 141])
tensor([ 18.4344, -20.9180, -16.0451,  ...,  -3.7429, -11.5620,  17.1256])


In [26]:
data = test_data
edge_index = data.edge_index
df_preds = df_data
output_pred = outputs
decision_threshold = 0.5
df = df_preds.copy()
print(edge_index.shape)
print(output_pred.shape)

torch.Size([2, 241757])
torch.Size([241757])


In [27]:

              
outputs_soft = torch.sigmoid(output_pred)
outputs_hard = (outputs_soft > decision_threshold).int()
print(outputs_hard)

connected_indices = edge_index[:, outputs_hard.bool()]#取出符合要求的边缘
output_save_pre = output_pred[outputs_hard.bool()]#取出符合要求的边缘置信度
print(connected_indices.shape)
print(output_save_pre.shape)

print(connected_indices)
print(output_save_pre)

print(edge_index)
print(outputs)

# print(edge_index[:,9])
# print(outputs[9])
# edge_index_see = pd.DataFrame(edge_index)
# outputs_see = pd.DataFrame(outputs)
# print(edge_index_see)
# print(outputs_see)

tensor([1, 0, 0,  ..., 0, 0, 1], dtype=torch.int32)
torch.Size([2, 20012])
torch.Size([20012])
tensor([[    0,     1,     2,  ..., 16137, 16139, 16146],
        [   53,    55,    54,  ..., 16167, 16154, 16171]])
tensor([18.4344, 17.6234, 15.4036,  ..., 16.0671, 17.4865, 17.1256])
tensor([[    0,     0,     0,  ..., 16146, 16146, 16146],
        [   53,    55,    74,  ..., 16154, 16164, 16171]])
tensor([ 18.4344, -20.9180, -16.0451,  ...,  -3.7429, -11.5620,  17.1256])


In [28]:

outputs_soft = torch.sigmoid(output_pred)
outputs_hard = (outputs_soft > decision_threshold).int()
print(outputs_hard)

connected_indices = edge_index[:, outputs_hard.bool()]#取出符合要求的边缘
output_save_pre = output_pred[outputs_hard.bool()]#取出符合要求的边缘置信度
print(connected_indices.shape)
print(output_save_pre.shape)

print(connected_indices)#change
print(connected_indices.shape)

label_fir = connected_indices[0,:]
label_sec = connected_indices[1,:]
label_sec_unique = np.unique(label_sec)




for label in label_sec_unique:
    label_cols = label_sec[label_sec == label]
    if label_cols.shape[0] > 1:
      
        ind_place_col = np.argwhere(label_sec==label)
      
        curr_position = df.loc[label,["centroid_row", "centroid_col"]].values   #第二行的位置
        
        
        x = label_fir[ind_place_col].numpy().squeeze()
        first_position = df.loc[x, ["centroid_row", "centroid_col"]].values     #第一行的位置

        distance = ((curr_position - first_position) ** 2).sum(axis=-1)
        nearest_cell = np.argmin(distance, axis=-1)
        node_remain = x[nearest_cell]     #要保留的ID


        delet_id = np.delete(x, np.where(x == node_remain))#需要改数据类型

        for del_id in delet_id:
            # 使用布尔索引删除连接
            delet_mask = np.logical_and(connected_indices[0, :] == del_id, connected_indices[1, :] == label)
            delete_indices = np.where(delet_mask)[0]
            delete_indices_test = np.where(delet_mask)
            connected_indices = np.delete(connected_indices, delete_indices, axis=1)
            output_save_pre = np.delete(output_save_pre,delete_indices)

        # 使用布尔索引删除连接
        # delet_mask = np.logical_and(connected_indices[0, :] == delet_id, connected_indices[1, :] == label)
        # # delet_mask = np.logical_and(connected_indices[0, :] == 0, connected_indices[1, :] == 53)
        # delete_indices = np.where(delet_mask)[0]
        # delete_indices_test = np.where(delet_mask)
        # connected_indices = np.delete(connected_indices, delete_indices, axis=1)
        
        



    
        # print('label:',label)





# 打印结果
print("Connected Indices (After Filtering):")
print(connected_indices)
print(connected_indices.shape)
print(output_save_pre)
print(output_save_pre.shape)

# torch.save(connected_indices, file1)
# torch.save(output_save_pre, file3)

# next_frame = df.loc[next_frame_ind, ["centroid_row", "centroid_col"]].values
# curr_node = df.loc[i, ["centroid_row", "centroid_col"]].values

# distance = ((next_frame - curr_node) ** 2).sum(axis=-1)
# nearest_cell = np.argmin(distance, axis=-1)
# # add to the array
# next_node_ind = next_frame_ind[nearest_cell]

tensor([[    0,     1,     2,  ..., 16137, 16139, 16146],
        [   53,    55,    54,  ..., 16167, 16154, 16171]])
torch.Size([2, 20012])
Connected Indices (After Filtering):
tensor([[    0,     1,     2,  ..., 16137, 16139, 16146],
        [   53,    55,    54,  ..., 16167, 16154, 16171]])
torch.Size([2, 15373])
tensor([18.4344, 17.6234, 15.4036,  ..., 16.0671, 17.4865, 17.1256])
torch.Size([15373])
